<a href="https://colab.research.google.com/github/41371113h-xian/114-1/blob/main/hw_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -q gradio pandas matplotlib python-dateutil gspread gspread_dataframe google-auth


In [17]:
# ===========================
# ToDo + Pomodoro — Gradio v4 全功能 + Google Sheets 同步（Colab 版）
# ===========================
import io
import os
import json
import time
import threading
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
from dateutil.parser import parse as dtparse

# Google Sheets 依賴
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default as google_auth_default

# -------- 常數 / 工具 --------
STATE_FILE = "todo_state.json"
DATE_FMT = "%Y-%m-%d"
DT_FMT = "%Y-%m-%d %H:%M:%S"

PRIORITIES = ["高", "中", "低"]
STATUSES = ["未開始", "進行中", "已完成", "已暫停"]

def now_str():
    return datetime.now().strftime(DT_FMT)

def safe_date_str(s):
    if not s:
        return ""
    try:
        return dtparse(str(s)).date().strftime(DATE_FMT)
    except Exception:
        return ""

def ensure_int(x, default=0):
    try:
        if isinstance(x, float) and pd.isna(x):
            return default
        return int(float(x))
    except Exception:
        return default

def parse_tags(s):
    s = (s or "").strip()
    if not s:
        return []
    raw = [x.strip() for x in s.replace(" ", " ").replace("　", " ").replace("，", ",").split(",")]
    return [x for x in raw if x]

def tags_to_str(tags):
    if not tags:
        return ""
    return ", ".join(tags)

# -------- 狀態 --------
def new_state():
    return {
        "tasks": [],
        "next_id": 1,
        "active": {
            "task_id": None,
            "running": False,
            "paused": False,
            "total_sec": 0,
            "left_sec": 0,
            "mode": "focus",  # focus / break_short / break_long
            "cycle_idx": 0,
            "stop_flag": False,
            "auto_next": True,
            "focus_min": 25,
            "short_break_min": 5,
            "long_break_min": 15,
            "long_every": 4,
            "sound_on": True
        },
        "settings": {
            "auto_complete_when_reach_est": True
        },
        "logs": []
    }

STATE = new_state()
LOCK = threading.Lock()

# -------- 永續化 --------
def save_state(path=STATE_FILE):
    with LOCK:
        with open(path, "w", encoding="utf-8") as f:
            json.dump(STATE, f, ensure_ascii=False, indent=2)

def load_state(path=STATE_FILE):
    if not os.path.exists(path):
        return False
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        for t in data.get("tasks", []):
            t["id"] = ensure_int(t.get("id"))
            t["est_pomodoros"] = ensure_int(t.get("est_pomodoros"), 1)
            t["used_pomodoros"] = ensure_int(t.get("used_pomodoros"))
            t["time_tracked_sec"] = ensure_int(t.get("time_tracked_sec"))
            t.setdefault("priority", "中")
            t.setdefault("status", "未開始")
            t.setdefault("desc", "")
            t["tags"] = t.get("tags", [])
        with LOCK:
            STATE.clear()
            STATE.update(data)
        return True
    except Exception as e:
        print("載入狀態失敗：", e)
        return False

load_state()

# -------- DataFrame / 視覺化 --------
def df_from_tasks(tasks=None):
    tasks = STATE["tasks"] if tasks is None else tasks
    cols = [
        "id","name","desc","status","priority","created_at","due_date","completed_at",
        "est_pomodoros","used_pomodoros","time_tracked_sec","tags"
    ]
    if not tasks:
        return pd.DataFrame(columns=cols)
    rows = []
    for t in tasks:
        rows.append({
            "id": t["id"],
            "name": t["name"],
            "desc": t.get("desc", ""),
            "status": t.get("status", "未開始"),
            "priority": t.get("priority", "中"),
            "created_at": t.get("created_at", ""),
            "due_date": t.get("due_date", ""),
            "completed_at": t.get("completed_at", ""),
            "est_pomodoros": t.get("est_pomodoros", 1),
            "used_pomodoros": t.get("used_pomodoros", 0),
            "time_tracked_sec": t.get("time_tracked_sec", 0),
            "tags": tags_to_str(t.get("tags", []))
        })
    return pd.DataFrame(rows, columns=cols)

def df_from_logs():
    cols = ["task_id","task_name","type","start","end","duration_sec","cycle_no"]
    if not STATE["logs"]:
        return pd.DataFrame(columns=cols)
    return pd.DataFrame(STATE["logs"], columns=cols)

def build_all_figures(df_tasks, df_logs):
    figs = []

    # 圖1：完成比例
    fig1 = plt.figure(figsize=(4.5, 4), dpi=120)
    if df_tasks.empty:
        plt.text(0.5, 0.5, "尚無任務", ha="center", va="center")
        plt.axis("off")
    else:
        stat = df_tasks["status"].value_counts()
        plt.pie(stat, labels=stat.index, autopct="%1.1f%%", startangle=90)
        plt.title("任務完成比例")
    figs.append(fig1)

    # 圖2：各任務番茄數
    fig2 = plt.figure(figsize=(6, 4), dpi=120)
    if df_tasks.empty:
        plt.text(0.5, 0.5, "尚無任務", ha="center", va="center")
        plt.axis("off")
    else:
        used = df_tasks.assign(used_pomodoros=pd.to_numeric(df_tasks["used_pomodoros"], errors="coerce").fillna(0))
        plt.bar(used["name"], used["used_pomodoros"])
        plt.xticks(rotation=45, ha="right")
        plt.title("各任務番茄數")
        plt.tight_layout()
    figs.append(fig2)

    # 圖3：每日番茄趨勢
    fig3 = plt.figure(figsize=(6, 4), dpi=120)
    if df_logs.empty:
        plt.text(0.5, 0.5, "尚無番茄紀錄", ha="center", va="center")
        plt.axis("off")
    else:
        fl = df_logs[df_logs["type"] == "focus"].copy()
        if fl.empty:
            plt.text(0.5, 0.5, "尚無番茄紀錄", ha="center", va="center")
            plt.axis("off")
        else:
            fl["day"] = pd.to_datetime(fl["end"]).dt.date
            trend = fl.groupby("day")["duration_sec"].sum().reset_index()
            trend["pomodoros"] = trend["duration_sec"] / 1500.0  # 25 分 = 1500 秒
            plt.plot(trend["day"], trend["pomodoros"], marker="o")
            plt.title("每日番茄趨勢（換算番茄數）")
            plt.xlabel("日期"); plt.ylabel("番茄數")
            plt.grid(True, alpha=0.3)
    figs.append(fig3)

    return figs

def refresh_all():
    df_tasks = df_from_tasks()
    df_logs = df_from_logs()
